In [170]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
import numpy as np

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33")

In [219]:
text = "dream"
hypothesis_template = "This word can be categorized as {}"
candidate_labels=["human", "animal", "plant", "vehicle", "food", "clothing", "body part", "furniture", "electronics", "item, tool or appliance", "manmade area", "natural area", "event", "feeling or emotion"]

result = zeroshot_classifier(text, candidate_labels, hypothesis_template=hypothesis_template)
label = result['labels'][0]
score = result['scores'][0]
result

{'sequence': 'dream',
 'labels': ['event',
  'feeling or emotion',
  'human',
  'manmade area',
  'vehicle',
  'natural area',
  'item, tool or appliance',
  'animal',
  'plant',
  'electronics',
  'body part',
  'clothing',
  'furniture',
  'food'],
 'scores': [0.5840466022491455,
  0.23004527390003204,
  0.0482359416782856,
  0.032118674367666245,
  0.031123893335461617,
  0.014237764291465282,
  0.013087446801364422,
  0.010301381349563599,
  0.008103832602500916,
  0.0074592577293515205,
  0.005932874511927366,
  0.005841523874551058,
  0.005505830980837345,
  0.003959634806960821]}

In [218]:
hypothesis_template = "This word can be categorized as {}"
candidate_labels=["human", "animal", "plant", "vehicle", "food", "clothing", "body part", "furniture", "electronics", "item, tool or appliance", "manmade area", "natural area", "event", "feeling or emotion"]


with open('laion_nouns.txt', 'r') as file:
    lines = file.readlines()
    nouns = [line.strip() for line in lines]

f = 'laion_nouns_classification.csv'

data = []

for noun in tqdm(nouns):
    result = zeroshot_classifier(noun, candidate_labels, hypothesis_template=hypothesis_template)
    data_n = [noun]
    for label in candidate_labels:
        idx = np.where(np.array(result['labels']) == label)[0][0]
        data_n.append(f"{result['scores'][idx]:.3f}")
    data.append(data_n)
    break

data = pd.DataFrame(data, columns=['noun'] + candidate_labels)
data.to_csv(f, index=False)

  0%|          | 0/6726 [00:02<?, ?it/s]
